COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: David Lu
#### Date: 24 Jan 2024

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [17]:
# API_KEY = 'DEMO_KEY'  # substitute your API key here
API_KEY = 'MQ8RytTcgwsCHIorzTANVhiyuM5Ghqn230HOkCSd'

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
# pprint(neo)

keys = list(neo.keys())
print(f'{keys[:8]}\n{keys[8:]}\n')
print(f'name = {neo["name"]}, id: {neo["id"]}, hazardous? {neo["is_potentially_hazardous_asteroid"]}\n')
pprint(neo['estimated_diameter'])
print()
pprint(neo['close_approach_data'][0])

118 Near Earth Objects found for week of 2023-01-15
['links', 'id', 'neo_reference_id', 'name', 'designation', 'nasa_jpl_url', 'absolute_magnitude_h', 'estimated_diameter']
['is_potentially_hazardous_asteroid', 'close_approach_data', 'orbital_data', 'is_sentry_object']

name = (2023 BM1), id: 54339874, hazardous? False

{'feet': {'estimated_diameter_max': 207.9816551205,
          'estimated_diameter_min': 93.0122237845},
 'kilometers': {'estimated_diameter_max': 0.0633928065,
                'estimated_diameter_min': 0.0283501249},
 'meters': {'estimated_diameter_max': 63.3928064522,
            'estimated_diameter_min': 28.3501249023},
 'miles': {'estimated_diameter_max': 0.0393904515,
           'estimated_diameter_min': 0.0176159455}}

{'close_approach_date': '1949-07-04',
 'close_approach_date_full': '1949-Jul-04 22:19',
 'epoch_date_close_approach': -646710060000,
 'miss_distance': {'astronomical': '0.419726307',
                   'kilometers': '62790161.51016609',
             

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [23]:
# Ex.1 your code here

class Asteroid:

    def __init__ (self, id, name, diameter, is_hazardous):  # Constructor

        self.id = id
        self.name = name
        self.diameter_m = diameter  # in metres
        self.is_hazardous = is_hazardous

    def __str__ (self):  # Informal string representation

        hazard_description = 'hazardous' if self.is_hazardous else 'non-hazardous'

        return (f'NEO {self.name} ({self.id}) is a {hazard_description} '
        f'asteroid with a diameter of {int(self.diameter_m)} m.')

    def __repr__ (self):  # Formal string representation

        return (f'name = {self.name}, id = {self.id}, diameter_m = {self.diameter_m}, '
        f'is_hazardous = {self.is_hazardous}')

    @classmethod
    def from_NEO (cls, neo_id):  # Factory method for exercise 2

        # Alternative constructor:
        # Constructs an Asteroid object from its reference ID

        record = get_neo(neo_id)  # Record for NEO

        id = neo_id
        name = record['name'][1:-1]
        is_hazardous = record['is_potentially_hazardous_asteroid']

        diameter = sum([record['estimated_diameter']['meters']
            [f'estimated_diameter_{measure}'] for measure in ('min', 'max')]) / 2

        return cls(id, name, diameter, is_hazardous)  # Construct Asteroid with these parameters

# ----------
# Some quick checks for exercise 1

astro_bob = Asteroid(1500, 'Bob', 20.0, False)

print(astro_bob)
print(repr(astro_bob))

assert astro_bob.id == 1500
assert astro_bob.name == 'Bob'
assert astro_bob.diameter_m == 20.0
assert astro_bob.is_hazardous == False


NEO Bob (1500) is a non-hazardous asteroid with a diameter of 20 m.
name = Bob, id = 1500, diameter_m = 20.0, is_hazardous = False


## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [24]:
# Ex. 2 your code here

# Method definition is included inside the Asteroid class definition
# (see code block for exercise 1). Here's an example of the method in action:

example = Asteroid.from_NEO('54339874')

print(example)
print(repr(example))

NEO 2023 BM1 (54339874) is a non-hazardous asteroid with a diameter of 45 m.
name = 2023 BM1, id = 54339874, diameter_m = 45.87146567725, is_hazardous = False


## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [ ]:
# Ex. 3 your code here

class CloseApproach:

    def __init__ (self, neo_obj, orbit_body, apr_date, miss_dist, rel_v):

        self.neo_object = neo_obj
        self.orbit_body = orbit_body
        self.approach_date = apr_date
        self.miss_distance_km = miss_dist   # km
        self.relative_velocity_kps = rel_v  # km/s (kilometres per second)

    def __str__ (self):

        return (f'NEO {self.neo_obj.name}/{self.neo_obj.id}: '
        f'Close approach with {self.orbit_body} on {self.approach_date} '
        f'at distance {int(self.miss_distance_km)} km and '
        f'relative velocity {self.relative_velocity_kps:.1f} km/s.')

    def __repr__ (self):

        return (f'neo = an {type(self.neo_obj).__name__}* object, '
        f'orbit_body = {self.orbit_body}, approach_date = {self.approach_date}'
        f'{type(self.approach_date)},\nmiss_distance_km = {self.miss_distance_km}, '
        f'relative_velocity_kps = {self.relative_velocity_kps}\n'
        f'*{type(self.neo_obj).__name__}: ({repr(self.neo_obj)})')

    @classmethod
    def from_record (cls, neo, apr_rec):

        assert isinstance(neo, Asteroid)  # neo is an object of Asteroid class

        orbit_body = apr_rec['orbiting_body']
        apr_date   = datetime.datetime.strptime(apr_rec['close_approach_date'],'%Y-%m-%d').date()
        miss_dist  = float(apr_rec['miss_distance']['kilometers'])
        r_velo     = float(apr_rec['relative_velocity']['kilometers_per_second'])

        return cls(neo, orbit_body, apr_date, miss_dist, r_velo)

# record = get_neo('54339874')
# first_close_approach = record['close_approach_data'][0]

# four = CloseApproach.from_record(example)

appppr = CloseApproach.from_record(example, neo['close_approach_data'][0])
print(appppr)
print(repr(appppr))

# fez = CloseApproach(astro_bob, 'Mars', '2004-02-03', '5000', '3.5')

# print(fez)
# print(repr(fez))

NEO 2023 BM1/54339874: Close approach with Earth on 1949-07-04 at distance 62790161 km and relative velocity 20.4 km/s.
neo = an Asteroid* object, orbit_body = Earth, approach_date = 1949-07-04<class 'datetime.date'>,
miss_distance_km = 62790161.51016609, relative_velocity_kps = 20.4193538659
*Asteroid: (name = 2023 BM1, id = 54339874, diameter_m = 45.87146567725, is_hazardous = False)


## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [ ]:
# Ex. 4 your code here

## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here